In [1]:
pip install -r requirements.txt

Ignoring pysqlite3-binary: markers 'sys_platform == "linux"' don't match your environment
  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.10.5-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached altair-5.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached anthropic-0.34.2-py3-none-any.whl.metadata (18 kB)
  Using cached anyio-4.5.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached asttokens-2.4.1-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached bcrypt-4.2.0-cp39-abi3-macosx_10_12_universal2.whl.metadata (9.6 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached build-1.2.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached ce

In [4]:
import os
import dotenv
from pathlib import Path
from dotenv import load_dotenv  # new addition

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

True

In [5]:
# Load docs

doc_paths = [
    "docs/test_rag.pdf",
    "docs/test_rag.docx",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [6]:
docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-09-15T19:40:36+02:00', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_siteid': 'eb25818e-5bd5-49bf-99de-53e3e7b42630', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_method': 'Standard', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_enabled': 'True', 'author': 'Domingo Domènech Enric (ERNI)', 'moddate': '2024-09-15T19:40:36+02:00', 'source': 'docs/test_rag.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='My favorite food is margarita pizza. \nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'docs/test_rag.docx'}, page_content='My favorite food is margarita pizza.\n\nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/quick-reference/release-notes', 'title': 'Release notes - Streamlit Docs', 'description': 'A changelog of highlights and fixes for ea

In [7]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

In [8]:
# Tokenize and load the documents to the vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

In [9]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [10]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [11]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


The latest version of Streamlit is 1.43.0, released on March 4, 2025.

In [12]:
import os
from langchain_openai import AzureChatOpenAI
import dotenv

dotenv.load_dotenv()

llm_stream = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZ_OPENAI_ENDPOINT"),
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key=os.getenv("AZ_OPENAI_API_KEY"),
    openai_api_type="azure",
    temperature=0.3,
    streaming=True,
)

prompt = "Tell me something about Azure"

for chunk in llm_stream.stream(prompt):
    print(chunk.content, end="", flush=True)

Certainly! Microsoft Azure is a comprehensive cloud computing platform developed by Microsoft. It provides a wide range of services and tools that enable businesses, developers, and organizations to build, deploy, and manage applications and services through Microsoft-managed data centers. Azure is one of the leading cloud service providers globally, competing with platforms like Amazon Web Services (AWS) and Google Cloud Platform (GCP).

### Key Features of Azure:
1. **Wide Range of Services**: Azure offers over 200 services, including computing, storage, networking, databases, artificial intelligence (AI), machine learning (ML), Internet of Things (IoT), analytics, DevOps, and more.

2. **Scalability**: Azure allows users to scale resources up or down based on demand, ensuring cost efficiency and performance optimization.

3. **Global Reach**: Azure has data centers located in regions across the globe, enabling users to deploy applications closer to their customers for better perform